# Operate the pipetools on iforge head node:

## import modules, set warning level, establish directory locations

In [1]:
import warnings
warnings.filterwarnings('ignore')
# pandas future quard

import os
import sys
from collections import OrderedDict

import numpy as np
import pandas as pd
import yaml

# clone_base = '/projects/bioinformatics/DEL'
clone_base = '/Users/mojo/git_clone/Mayomics-Research'
config_base = clone_base

wdl_directory = os.path.join(clone_base,'MayomicsVC/src/wdl')

os.listdir(wdl_directory)

['SomaticVC',
 'DeliveryOfSomaticVC',
 'GermlineMasterWorkflow.wdl',
 'Alignment',
 'SomaticMasterWorkflow.wdl',
 'SomaticVCStandaloneWorkflow.wdl',
 'DeliveryOfHaplotyperVC',
 'DeliveryOfAlignment',
 'HaplotyperVC']

## Parse the "wdl_directory", read all ".wdl" files, get types used in the "task" sections

In [4]:
def get_task_types(wdl_directory=None):
    """ Usage: types_list = get_task_types(wdl_directory)
    get the unique list of types in the task section of all wdl files in the dir tree 
    """
    if not wdl_directory is None and os.path.isdir(wdl_directory):
        wdl_directory = wdl_directory
    else:
        wdl_directory = os.getcwd()

    skip_line_chars = ['#', '<', '>']
    switch_ON_words = ['task']
    switch_OFF_words = ['command', 'runtime', 'output']
    add_words_ON = False

    type_words = {}
    for this_dir, dirs, files in os.walk(wdl_directory):
        for file in files:
            if file[-4:] == '.wdl':
                add_words_ON = False
                full_filename = os.path.join(this_dir, file)
                
                with open(full_filename, 'r') as fh:
                    lines = fh.readlines()
                    
                for line in lines:
                    l = line.strip()
                    if len(l) > 0 and not l[0] in skip_line_chars:
                        type_word = l.split()[0]
                        
                        if type_word in switch_OFF_words:
                            add_words_ON = False
                            
                        if add_words_ON == True:
                            if type_word in type_words:
                                type_words[type_word] += 1
                            else:
                                type_words[type_word] = 1
                            
                        if type_word in switch_ON_words:
                            add_words_ON = True

    return type_words

types_dict = get_task_types(wdl_directory)
print('There are %i types in all the wdl files:\n'%(len(types_dict)))
for wurd, wurd_count in types_dict.items():
    print('%20s = %i'%(wurd, wurd_count))

There are 5 types in all the wdl files:

                File = 123
              String = 114
             Boolean = 2
         Array[File] = 2
               File? = 1


## Walk the 'wdl' directory, parse all wdl files, build a dictionary -- variable names: type

In [5]:
def get_wdl_variables_dict(wdl_directory=None):
    """ Usage  config_orderd_dict = get_wdl_variables_dict(wdl_directory)
    Get the sorted list of variables and their types from the wdl files in a directory tree
    
    Args:
        wdl_directory:       (default to run directory if not valid directory name)
        
    Returns:
        config_ordered_dict: python OrderedDict of  variable_name: "type_name"
        
    """
    # Get the complete config.txt file:
    skip_line_chars = ['#', '<', '>']
    task_types_list = sorted(['Array[File]', 'File', 'File?', 'Boolean', 'String'])
    config_vars_dict = {}
    for this_dir, dirs, files in os.walk(wdl_directory):
        for file in files:
            if file[-4:] == '.wdl' and file[0] != '.':
                full_filename = os.path.join(this_dir, file)
                with open(full_filename, 'r') as fh:
                    lines = fh.readlines()
                for line in lines:
                    l = line.strip()
                    if len(l) > 0 and not l[0] in skip_line_chars:
                        line_words_list = l.split()
                        first_word = line_words_list[0]
                        if first_word in task_types_list:
                            second_word = line_words_list[1]
                            if not second_word in config_vars_dict.keys():
                                config_vars_dict[second_word] = '"' + first_word + '"'
                                
                            elif first_word != config_vars_dict[second_word][1:-1]:
                                bugger = config_vars_dict[second_word][:-1]
                                config_vars_dict[second_word] = bugger + ', ' + first_word  + '"'
                                
    config_od = OrderedDict()
    for k, v in sorted(config_vars_dict.items()):
        config_od[k] = v

    return config_od

config_vars_dict_ret = get_wdl_variables_dict(wdl_directory)
print('found %i variables'%(len(config_vars_dict_ret)))
config_od = OrderedDict()
for k, v in sorted(config_vars_dict_ret.items()):
    config_od[k] = v
    
for k, v in config_od.items():
    print('%30s: %20s'%(k,v))


found 158 variables
                      Adapters:               "File"
               AlignEnvProfile:               "File"
             AlignHardMemLimit:             "String"
               AlignOutputBais:        "Array[File]"
               AlignOutputBams:        "Array[File]"
             AlignSoftMemLimit:             "String"
               AlignmentScript:               "File"
                  AnnotateText:             "String"
         BWAExtraOptionsString:             "String"
                  BashPreamble:               "File"
           BashSharedFunctions:               "File"
                      Bcftools:             "String"
    BcftoolsExtraOptionsString:             "String"
                         Bgzip:             "String"
                          Bqsr:            "Boolean"
                BqsrEnvProfile:               "File"
              BqsrHardMemLimit:             "String"
                BqsrKnownSites:             "String"
                    BqsrSc

## Walk the 'wdl' directories, find the functional parts, build a comparison spreadsheet

In [7]:
def get_wdl_files_spreadsheet(wdl_directory=None):
    """ wdl_files_spreadsheet = get_wdl_files_spreadsheet(wdl_directory)
    get the section statistics spreadsheet for a directory tree of wdl files 
    
    Args:
        wdl_directory:       (default to run directory if not valid directory name)
        
    Returns:
        wdl_files_spreadsheet: pandas dataframe with rows of wdl filenames, cols of wdl keywords

    """
    if not wdl_directory is None and os.path.isdir(wdl_directory):
        may_dir = wdl_directory
    else:
        may_dir = os.getcwd()
    
    dir_name_stripper, _ = os.path.split(wdl_directory)

    prefix_ignores = ['.', '_']
    
    # assemble dictionary - wdl file names: full path  -------- Build empty pandas dataframe
    mdl_dict = {}
    for dir_name, dir_list, files_list in os.walk(may_dir):
        if len(files_list) > 0:
            for file_name in files_list:
                if file_name[-4:] == '.wdl' and file_name[0] not in prefix_ignores:
                    mdl_dict[file_name] = os.path.join(dir_name, file_name)
    wdl_df = pd.DataFrame.from_dict(mdl_dict, orient='index', columns=['src_path'])
    wdl_df.index.name = 'WDL file'
    
    # define the rest of the columns in terms of wdl keywords
    srch_dict = OrderedDict([('imports','import'), 
                             ('calls','call'), 
                             ('workflows','workflow'), 
                             ('tasks','task'), 
                             ('inputs','input'), 
                             ('outputs','output'), 
                             ('commands', 'command')])
    for col_name, _ in srch_dict.items():
        wdl_df[col_name] = 0

    # for each wdl file: count keywords, get top path ---------- fill pandas dataframe
    for name, row in wdl_df.iterrows():
        """ using row.src_path to open files and name to refer to row in dataframe """
        lines = []
        try:
            with open(row.src_path, 'r') as fh:
                lines = fh.readlines()
        except:
            print('Fails to open:\n', row.src_path)
            pass
        
        # count & insert occurrence of keywords
        if len(lines) > 0:
            for l in lines:
                for col_name, key_word in srch_dict.items():
                    if key_word in l:
                        wdl_df.loc[name,col_name] += 1

        # replace the full path name with top path name for readability
        full_path, _ = os.path.split(mdl_dict[name])
        wdl_df['src_path'].loc[name] = full_path.replace(dir_name_stripper, '..')
        
    return wdl_df

wdl_df = get_wdl_files_spreadsheet(wdl_directory)
print(wdl_df.shape)
wdl_df

(36, 8)


src_path  \
WDL file                                                                       
GermlineMasterWorkflow.wdl                                            ../wdl   
SomaticMasterWorkflow.wdl                                             ../wdl   
SomaticVCStandaloneWorkflow.wdl                                       ../wdl   
manta.wdl                                             ../wdl/SomaticVC/Tasks   
strelka.wdl                                           ../wdl/SomaticVC/Tasks   
mutect.wdl                                            ../wdl/SomaticVC/Tasks   
combine_variants.wdl                                  ../wdl/SomaticVC/Tasks   
Teststrelka.wdl                                   ../wdl/SomaticVC/TestTasks   
Testcombine_variants.wdl                          ../wdl/SomaticVC/TestTasks   
Testmutect.wdl                                    ../wdl/SomaticVC/TestTasks   
deliver_SomaticVC.wdl                       ../wdl/DeliveryOfSomaticVC/Tasks   
Testdeliver_somatic.wdl                 ../wdl/DeliveryOfSomaticVC/TestTasks   
trim_sequences.wdl                                    ../wdl/Alignment/Tasks   
dedup.wdl                                             ../wdl/Alignment/Tasks   
merge_aligned_bam.wdl                                 ../wdl/Alignment/Tasks   
alignment.wdl                                         ../wdl/Alignment/Tasks   
Testdedup.wdl                                     ../wdl/Alignment/TestTasks   
Runtrim_sequences.wdl                             ../wdl/Alignment/TestTasks   
Runalignment.wdl                                  ../wdl/Alignment/TestTasks   
TestAlignment.wdl                                  ../wdl/Alignment/Workflow   
deliver_HaplotyperVC.wdl                 ../wdl/DeliveryOfHaplotyperVC/Tasks   
Testdeliver_haplotyperVC.wdl         ../wdl/DeliveryOfHaplotyperVC/TestTasks   
DeliverHaplotyperVC_ForWorkflow.wdl   ../wdl/DeliveryOfHaplotyperVC/Workflow   
deliver_alignment.wdl                       ../wdl/DeliveryOfAlignment/Tasks   
Testdeliver_alignment.wdl               ../wdl/DeliveryOfAlignment/TestTasks   
DeliverAlignment_ForWorkflow.wdl         ../wdl/DeliveryOfAlignment/Workflow   
TestDeliverAlignment.wdl                 ../wdl/DeliveryOfAlignment/Workflow   
realignment.wdl                                    ../wdl/HaplotyperVC/Tasks   
haplotyper.wdl                                     ../wdl/HaplotyperVC/Tasks   
bqsr.wdl                                           ../wdl/HaplotyperVC/Tasks   
vqsr.wdl                                           ../wdl/HaplotyperVC/Tasks   
Testrealignment.wdl                            ../wdl/HaplotyperVC/TestTasks   
Testhaplotyper.wdl                             ../wdl/HaplotyperVC/TestTasks   
Testbqsr.wdl                                   ../wdl/HaplotyperVC/TestTasks   
Testvqsr.wdl                                   ../wdl/HaplotyperVC/TestTasks   
TestHaplotyperVC.wdl                            ../wdl/HaplotyperVC/Workflow   

                                     imports  calls  workflows  tasks  inputs  \
WDL file                                                                        
GermlineMasterWorkflow.wdl                10     10          1      0      10   
SomaticMasterWorkflow.wdl                 11     18          4      0      17   
SomaticVCStandaloneWorkflow.wdl            5      7          1      0       6   
manta.wdl                                  0      1          0      1       0   
strelka.wdl                                0      1          0      1       0   
mutect.wdl                                 0      1          0      1       0   
combine_variants.wdl                       0      2          0      2       0   
Teststrelka.wdl                            1      2          1      0       0   
Testcombine_variants.wdl                   1      2          1      0       0   
Testmutect.wdl                             1      2          1      0       0   
deliver_SomaticVC.wdl                      0      0    

## Read a ".json" file, build a spreadsheet from a dictionary of variables (2-do add the types)

In [8]:
import yaml

def read_json_to_df(full_filename):
    """ Usage: json_df = read_json_to_df(full_filename)
    read a json file into a spreadsheet
    """
    if os.path.isfile(full_filename) != True:
        run_parameters = {}
    else:
        with open(full_filename, 'r') as file_handle:
            run_parameters = yaml.load(file_handle)
        if not isinstance(run_parameters, dict):
            run_parameters = {}
            
    json_df = pd.DataFrame.from_dict(run_parameters, orient='index', columns=['Variable Value'])
    json_df.index.name = 'Variable Name'
    
    return json_df

config_base = clone_base
try_dir = os.path.join(config_base, 'Jsons/GermlineMasterWorkflow.FilledIn.json')
jojo_df = read_json_to_df(try_dir)
print('spreadsheet_size = ', jojo_df.shape)
jojo_df

spreadsheet_size =  (0, 1)


Empty DataFrame
Columns: [Variable Value]
Index: []

## Example cell how to develop a python test script
    * (cell below is not actually code)
    * the cell below would be writte to the desired spot
    * could be run from the command line or in the notebook

```python
%%writefile check_test_ouput.py
"""
check_test_ouput.py
Usage:
python ~/python/check_test_ouput.py -d /projects/mgc/Project_1/DEL/MVP/cromwell-executions/GermlineMasterWF/
or 
python ~/python/check_test_ouput.py -d `pwd`

check return codes and std_out in directory tree
"""

import os
import argparse

good_return_codes_list = ['0', '0\n']
file_names_to_show_list = ['rc', 'stdout', 'stderr']

def check_rc_codes(x_directory=None, show_stds=False):
    if not x_directory is None and os.path.isdir(x_directory):
        dir_tree_root = x_directory
    else:
        dir_tree_root = os.getcwd()
    root_trim_str, _ = os.path.split(dir_tree_root)
    for dir_name, dir_list, files_list in os.walk(dir_tree_root):
        for filename in files_list:
            if filename in file_names_to_show_list:
                full_filename = os.path.join(dir_name, filename)
                with open(full_filename, 'r') as fh:
                    lines = fh.readlines()
                if len(lines) <= 0:
                    continue
                elif filename == 'rc':
                    if lines[0] in good_return_codes_list:
                        top_dir = dir_name.replace(root_trim_str, '..')
                        print('good rc:  %s'%(top_dir))
                    else:
                        print('\n\tBad Dog! Bad Dog!')
                        print('code = %s in \n%s\n'%(str(lines[0]).strip(), full_filename))
                elif show_stds:
                    # opportunity to reformat for readability
                    for line in lines:
                        print(line)
                    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('-d', type=str)
    parser.add_argument('-show_stds', nargs="?", const=False, type=bool)
    args = parser.parse_args()
    check_rc_codes(args.d, args.show_stds)
```